In [4]:
from scipy._lib._test_ccallback import test_get_data_capsule

from models.graph_transformers.SAT.sat import models
from models.graph_transformers.SAT.sat import layers
import torch
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader


def ptr_to_complete_edge_index(ptr):
    from_lists = [torch.arange(ptr[i], ptr[i + 1]).repeat_interleave(ptr[i + 1] - ptr[i]) for i in range(len(ptr) - 1)]
    to_lists = [torch.arange(ptr[i], ptr[i + 1]).repeat(ptr[i + 1] - ptr[i]) for i in range(len(ptr) - 1)]
    combined_complete_edge_index = torch.vstack((torch.cat(from_lists, dim=0), torch.cat(to_lists, dim=0)))
    return combined_complete_edge_index
test_data = torch.FloatTensor([1., 2., 3., 4., 5.]).unsqueeze(1)
edge_index = torch.LongTensor([[0,1,2],[1, 2, 3]])

test_graph = Data(x=test_data, edge_index=edge_index)
test_data2 = torch.FloatTensor([-1., -2., -3., -4., 10]).unsqueeze(1)
edge_index2 = torch.LongTensor([[0,1],[1, 2]])
test_graph2 = Data(x=test_data2, edge_index=edge_index2)
loader = iter(DataLoader([test_graph, test_graph2], batch_size=2))
batch = next(loader)


batch.complete_edge_index = torch.LongTensor([[0,1,5],[1,2,6]])
# batch.complete_edge_index = ptr_to_complete_edge_index(batch.ptr)


model = models.GraphTransformer(in_size=1,
                                num_class=1,
                                in_embed=False,
                                d_model=3,
                                dim_feedforward=2,
                                num_heads=1,
                                num_layers=1,
                                abs_pe=False,
                                abs_pe_dim=None,

                                use_edge_attr=False,
                                num_edge_features=0,
                                dropout=0.,
                                k_hop=2)


#model(batch)


KeyboardInterrupt



In [ ]:
amrlayer = layers.AttentionAMR(1, 0, 1, 0, True).cuda()

In [ ]:
batch.ptr

In [ ]:
edge_index_source = torch.LongTensor([[i for i in torch.arange(batch.edge_index.shape[1])], [batch.edge_index[0][i] for i in torch.arange(batch.edge_index.shape[1])]]).cuda()
edge_index_target = torch.LongTensor([[i for i in torch.arange(batch.edge_index.shape[1])], [batch.edge_index[1][i] for i in torch.arange(batch.edge_index.shape[1])]]).cuda()

amrlayer(batch.x.cuda(), batch.x.cuda(), batch.edge_index.cuda(), edge_index_source, edge_index_target, ptr=batch.ptr.cuda())

In [ ]:
embed = torch.nn.Linear(1000, 128)

In [ ]:
import pickle
with open("data/hol4/data/torch_graph_dict.pk", "rb") as f:
    tg = pickle.load(f)
test_graph_1 = tg[list(tg.keys())[0]]
test_graph_1
test_graph_2 = tg[list(tg.keys())[2]]
test_graph_2
loader = iter(DataLoader([test_graph_1, test_graph_2], batch_size=2))
batch = next(loader)

In [ ]:

from models.graph_transformers.SAT.sat import layers
amrlayer = layers.AttentionAMR(128,0,2,0,True)

x = embed(batch.x)

out_s, out_t = amrlayer(x, x, batch.edge_index)

In [ ]:
amrlayer(out_s, out_t, batch.edge_index)

In [ ]:
batch.edge_index

In [ ]:

batch.edge_attr = batch.edge_attr.long()
model = models.AMRTransformer(in_size=1000,num_class=0,d_model=128,num_heads=4,dim_feedforward=128,dropout=0.1,use_edge_attr=True,num_layers=1,layer_norm=True,in_embed=False).cuda()
model(batch.cuda())

In [ ]:
torch.tensor([1.,2.,3.,4.,5.]).to(torch.tensor([1.,1.,2.,5,5234,643,2345]))

In [ ]:
batch.x

In [ ]:
import torch
from models.graph_transformers.SAT.sat.layers import AttentionRelations
model = AttentionRelations(embed_dim=8, edge_dim=32)
x = torch.rand(11, 8)
edge_index = torch.LongTensor([[0,1,2,3,4,9,8,7,6,8],[4,3,2,1,0,7,8,8,6,6]])
edge_attr = torch.LongTensor([0,0,1,3,0,1,2,3,2,1])
softmax_idx = [5]
model(x, edge_index, softmax_idx, edge_attr)

In [ ]:
torch.tensor_split(x, [0,5,10])

In [ ]:
'''
Test for dropout memory issues using TransformerEncoderLayer
'''

In [1]:
import torch
from models.transformer_encoder_model import TransformerEmbedding


In [2]:

test_input = torch.rand((1024,32,128)).cuda()
# test_input = torch.randint(100, (1024,32)).cuda()
# test_input.shape


# test_model = torch.nn.TransformerEncoderLayer(128,4,512,dropout=0).cuda()




encoder_layers = torch.nn.TransformerEncoderLayer(128, 4, 2, dropout=0.0)

test_model = torch.nn.TransformerEncoder(encoder_layers, 2).cuda()



# test_model = TransformerEmbedding(100,128,4,512,2,dropout=0.0,in_embed=False,global_pool=False).cuda()

op = torch.optim.Adam(test_model.parameters())
op.zero_grad()

out = test_model(test_input)
loss = torch.sum(out)
loss.backward()

op.step()

In [ ]:
# using standard TransformerEncoderLayer API, get very large memory increase when using dropout

In [4]:
import pymongo
import torch

In [5]:
class DatabaseDataset(torch.utils.data.Dataset):
    def __init__(self, query, batch_size, collection: str, database: str):
        self.batch_size = batch_size

        client = pymongo.MongoClient()
        self.db = client[database]
        self.collection = self.db[collection]
        self.query = query

        # Or non-approximate method, if the approximate method
        # returns smaller number of items you should be fine
        # self.length = self.db.estimated_document_count()

        self.cursor = None

    def __enter__(self):
        # Ensure that this find returns the same order of query every time
        # If not, you might get duplicated data
        # It is rather unlikely (depending on batch size), shouldn't be a problem
        # for 20 million samples anyway
        self.cursor = self.db.find(self.query)
        return self

    def shuffle(self):
        # Find a way to shuffle data so it is returned in different order
        # If that happens out of the box you might be fine without it actually
        pass

    def __exit__(self, *_, **__):
        # Or anything else how to close the connection
        self.cursor.close()

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, index):
        # Read takes long, hence if you can load a batch of documents it should speed things up
        examples = self.cursor[index * self.batch_size : (index + 1) * self.batch_size]

        # Do something with this data
        ...
        # Return the whole batch
        return data, labels

In [ ]:
train_cursor = split_collection.aggregate([{"$match": {"split": "train"}}, {"$sample": {"size": 10000000}}])
batches = get_batches(train_cursor, self.batch_size)
